In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def MetaLink(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_D1, z, y)
    elif x == "H8":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H8, z, y)
    elif x == "H4":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H4, z, y)
    elif x == "H3":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H3, z, y)
    elif x == "H2":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H2, z, y)
    elif x == "H1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H1, z, y)
    elif x == "M30":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M30, z, y)
    elif x == "M15":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M15, z, y)
    elif x == "M5":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M5, z, y)
    elif x == "M1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M1, z, y)
    return data

In [4]:
def MetaLink_range(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_D1, y, z)
    elif x == "H8":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H8, y, z)
    elif x == "H4":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H4, y, z)
    elif x == "H3":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H3, y, z)
    elif x == "H2":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H2, y, z)
    elif x == "H1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H1, y, z)
    elif x == "M30":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M30, y, z)
    elif x == "M15":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M15, y, z)
    elif x == "M5":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M5, y, z)
    elif x == "M1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M1, y, z)
    return data

In [5]:
def MetaLink_pos(p,x,y):
    if x == "D1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_D1, 0, y)
    elif x == "H8":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H8, 0, y)
    elif x == "H4":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H4, 0, y)
    elif x == "H3":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H3, 0, y)
    elif x == "H2":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H2, 0, y)
    elif x == "H1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H1, 0, y)
    elif x == "M30":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M30, 0, y)
    elif x == "M15":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M15, 0, y)
    elif x == "M5":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M5, 0, y)
    elif x == "M1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M1, 0, y)
    return data

In [6]:
def Pip_Screen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            D = d*10000
        elif base == "XAU":
            D = d*100
        elif base == "BTC":
            D = d*10
        elif base == "BCH":
            D = d*100
        elif base == "ETH":
            D = d*10
        elif base == "DSH":
            D = d*100
        elif base == "LTC":
            D = d*100
        elif base == "XMR":
            D = d*100
        elif base == "ZEC":
            D = d*100
        elif base == "ETC":
            D = d*1000
        elif base == "XRP":
            D = d*10000
        else:
            D = d*10000
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            D = d*10
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        D = d*10000
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        D = d*10000
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        D = d*10000
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        D = d*10000
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        D = d*10000
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        D = d*100
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        D = d*10000
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        D = d*1000
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        D = d*1000
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        D = d*10000
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        D = d*1000
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        D = d*1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d*10
    elif quote == "ICrude":
        exchange = 1.00
        D = d*10
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        D = d*10000
        
    return D

In [7]:
def Pip_ReverseScreen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        if base == "XAG":
            D = d/10000
        elif base == "XAU":
            D = d/100
        elif base == "BTC":
            D = d/10
        elif base == "BCH":
            D = d/100
        elif base == "ETH":
            D = d/10
        elif base == "DSH":
            D = d/100
        elif base == "LTC":
            D = d/100
        elif base == "XMR":
            D = d/100
        elif base == "ZEC":
            D = d/100
        elif base == "ETC":
            D = d/1000
        elif base == "XRP":
            D = d/10000
        else:
            D = d/10000
    if quote == "USDt":
        if base == "BTC":
            D = d/10
    elif quote == "AUD":
        D = d/10000
    elif quote == "CAD":
        D = d/10000
    elif quote == "CHF":
        D = d/10000
    elif quote == "EUR": 
        D = d/10000
    elif quote == "GBP":
        D = d/10000
    elif quote == "JPY":
        D = d/100
    elif quote == "NZD":
        D = d/10000
    elif quote == 'NOK':
        D = d/1000
    elif quote == 'SEK':
        D = d/1000
    elif quote == 'SGD':
        D = d/10000
    elif quote == 'DKK':
        D = d/1000
    elif quote == 'MXN':
        D = d/1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d/10
    elif quote == "ICrude":
        exchange = 1.00
        D = d/10
    elif quote == "TRY":
        D = d/10000
        
    return D

In [8]:
def levelgrab(pair):
    
    levslist = []
    
    for p in pair:
        
        S = MetaLink_pos(p,"H1",1000)
        SCM = np.average(abs(S['close'] - S['open']))
        y = savgol_filter(S['close'], 21, 3)
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)           
        levs = tuple(levs)
        levslist.append(levs)

    return levslist, SCM

In [9]:
def levelwheel(levs, SCM):
    
    mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
    while mindist < SCM:
        i = 0
        h = -1
        j = 1
        for c in range(0, len(levs)):
            if j == len(levs):
                j = -1
            if abs(levs[i]-levs[h]) < SCM:
                levs[i] = 0
            elif abs(levs[i]-levs[j]) < SCM:
                levs[i] = levs[i] - abs(levs[i]-levs[j])/2
            i += 1
            h += 1
            j += 1
        levs = [v for v in levs if v != 0]
        mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])

    levs = tuple(levs)

    return levs

In [10]:
def Trade_Check(current_trade):
    
    trade_flag = 1
    list_check = 1
    
    to_date=dt.now()
    delta1=timedelta(
        days=0,
        seconds=0,
        microseconds=0,
        milliseconds=0,
        minutes=0,
        hours=1,
        weeks=0
    )
    
    try:
        deals = mt5.history_orders_get(from_date, to_date)
        df = pd.DataFrame(list(deals),columns=deals[0]._asdict().keys())
        df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
        df = df.drop(columns=['ticket', 'magic', 'position_by_id', 'reason', 'time_setup', 'time_setup_msc', 'time_done_msc', 'time_expiration', 'type_time', 'type_filling', 'external_id'])
        aggregation_functions = {'time_done': 'first', 'type': 'first', 'state': 'count', 'volume_initial': 'first', 'volume_current': 'first', 'price_open': 'first', 'sl': 'first', 'tp': 'first', 'price_current': 'first', 'price_stoplimit': 'last', 'symbol': 'first', 'comment': 'first'}
        df = df.groupby(df['position_id']).aggregate(aggregation_functions)
        block = pd.DataFrame(df, columns=['comment', 'state', 'symbol', 'type', 'time_done', 'price_current', 'tp', 'sl'])
        block = block.sort_values(by=['comment', 'time_done'])
        testa = pd.DataFrame(block[(block.comment == "BRUTUS gauge") & (block.state == 2)])
        testa = testa.drop(columns=['comment','state'])
        testa['type'] = np.where(testa['type'] == 1, "SHORT","LONG")
        testa.columns = ['symbol', 'type', 'time', 'entry', 'tp', 'sl']
        testa = testa.set_index('time')
        testa = testa.tz_localize('Etc/GMT-7')
        testa = testa.tz_convert('UTC')
        trade_symb = list(testa['symbol'])
        trade_type = list(testa['type'])
        trades = list(zip(trade_symb,trade_type))

    except ValueError:
        list_check = 0
    
    if list_check == 1:
        for t in trades:
            if t == current_trade:
                trade_flag = 0
        
    return trade_flag

In [11]:
def positioner(a,b,c,d,e):
    
    riskusd = 50.00        

    base, quote = a[:3], a[3:]
    
    ro = 2

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        elif base == "XAU":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BCH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETH":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "DSH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "LTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "XMR":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ZEC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETC":
            stpips = abs(e - c)*1000
            lot = round(((riskusd*exchange)/10)/10/stpips,ro)
        elif base == "XRP":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        else:
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/stpips,ro)
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        stpips = abs(e - c)*100
        lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == "entCrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "ICrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS v3",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS v3",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
            
    lotflag = "GREEN"
    if lot == 0:
        lotflag = "RED"
    
    return request, lotflag

In [12]:
def brutus(pair,levslist,SCM):

    ia = 0
    for p in pair:         
        A = MetaLink_pos(p,"M5",80)
        P = Pip_Screen(p,A['close'])
        S = savgol_filter(A['close'], 13, 3)
        slope, intercept, r_value, p_value, std_err = stats.linregress(range(len(P)),P)
        ST = slope
        ST_theta = abs(math.degrees(math.atan(ST)))
        if ST_theta > 20:
            if ST > 0:
                try:
                    B = list(S)
                    break_peak = max(B)
                    bp_index = B.index(break_peak) 
                    B2 = B[:bp_index]
                    break_spawn = min(B2)
                    bs_index = B.index(break_spawn)
                    levels = list(levslist[ia])
                    Q = list(break_peak - levels)
                    Qi = [i for i in Q if i > 0]
                    Qi_min = min(Qi)
                    Qi_index = Q.index(Qi_min)
                    LVL = levels[Qi_index]
                    CM_avg = np.average(abs(A['close']-A['open']))
                    CM_std = np.std(abs(A['close']-A['open']))
                    if break_peak > LVL + CM_avg + CM_std:
                        if break_spawn < LVL - CM_avg - CM_std:
                            break_check = 1
                        else:
                            break_check = 0
                    else:
                        break_check = 0
                except ValueError:
                    break_check = 0
                if break_check == 1:
                    gap = break_peak - LVL
                    retrace = break_peak - A['close'][-2]
                    if retrace/gap >= 0.60:
                        retrace_check = 1
                    else:
                        retrace_check = 0
                else:
                    retrace_check = 0
                if retrace_check == 1:
                    C = S[-7:]
                    C1 = []
                    x = 0
                    for c in C:
                        if x == 0:
                            C1.append(0)
                        else:
                            cc = C[x] - C[x-1]
                            C1.append(cc)
                        x += 1
                    C2 = []
                    x = 0
                    for c in C1:
                        if x == 0:
                            C2.append(0)
                        else:
                            ccc = C1[x] - C1[x-1]
                            C2.append(ccc)
                        x += 1
                    if C2[-1] > 0:
                        print("Delta2Y #1 :  1")
                        if C2[-2] > 0:
                            print("Delta2Y #2 :  1")
                            if C2[-3] > 0:
                                print("Delta2Y #3 :  1")
                                delta_check = 1
                            else:
                                delta_check = 0
                        else:
                            delta_check = 0
                    else:
                        delta_check = 0
                else:
                    delta_check = 0
                if delta_check == 1:
                    entry = A['close'][-2]
                    takeprofit = entry + gap*1.5
                    stoploss = entry - (takeprofit - entry)/2
                    request, lotflag = positioner(p,"LONG",entry,takeprofit,stoploss)
                    print(p,"LONG",lotflag)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Short","   TP:  ",takeprofit,"   SL:  ",stoploss,)
            elif ST > 0:
                try:
                    B = list(S)
                    break_peak = min(B)
                    bp_index = B.index(break_peak) 
                    B2 = B[:bp_index]
                    break_spawn = max(B2)
                    bs_index = B.index(break_spawn)
                    levels = list(levslist[ia])
                    Q = list(break_peak - levels)
                    Qi = [i for i in Q if i < 0]
                    Qi_min = max(Qi)
                    Qi_index = Q.index(Qi_min)
                    LVL = levels[Qi_index]
                    CM_avg = np.average(abs(A['close']-A['open']))
                    CM_std = np.std(abs(A['close']-A['open']))
                    if break_peak < LVL - CM_avg - CM_std:
                        if break_spawn > LVL + CM_avg + CM_std:
                            break_check = 1
                        else:
                            break_check = 0
                    else:
                        break_check = 0
                except ValueError:
                    break_check = 0
                if break_check == 1:
                    gap = LVL - break_peak
                    retrace = A['close'][-2] - break_peak
                    if retrace/gap >= 0.60:
                        retrace_check = 1
                    else:
                        retrace_check = 0
                else:
                    retrace_check = 0
                if retrace_check == 1:
                    C = S[-7:]
                    C1 = []
                    x = 0
                    for c in C:
                        if x == 0:
                            C1.append(0)
                        else:
                            cc = C[x] - C[x-1]
                            C1.append(cc)
                        x += 1
                    C2 = []
                    x = 0
                    for c in C1:
                        if x == 0:
                            C2.append(0)
                        else:
                            ccc = C1[x] - C1[x-1]
                            C2.append(ccc)
                        x += 1
                    if C2[-1] < 0:
                        print("Delta2Y #1 :  1")
                        if C2[-2] < 0:
                            print("Delta2Y #2 :  1")
                            if C2[-3] < 0:
                                print("Delta2Y #3 :  1")
                                delta_check = 1
                            else:
                                delta_check = 0
                        else:
                            delta_check = 0
                    else:
                        delta_check = 0
                else:
                    delta_check = 0
                if delta_check == 1:
                    entry = A['close'][-2]
                    takeprofit = entry - gap*1.5
                    stoploss = entry + (entry - takeprofit)/2
                    request, lotflag = positioner(p,"SHORT",entry,takeprofit,stoploss)
                    print(p,"SHORT",lotflag)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Short","   TP:  ",takeprofit,"   SL:  ",stoploss,)
        ia += 1

In [13]:
print("Initializing...")

pair = []
symbols=mt5.symbols_get()
for s in symbols:
    pair.append(s.name)

levslist, SCM = levelgrab(pair)

t2 = 0
while True:
    Drates = MetaLink_pos("EURUSD","M5",1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            brutus(pair,levslist,SCM)
            print("...Analysis Complete")
    t2 = t
    time.sleep(10)

Initializing...
Analyzing Markets...
...Analysis Complete
Analyzing Markets...
...Analysis Complete
Analyzing Markets...
Delta2Y #1 :  1
Delta2Y #1 :  1
Delta2Y #2 :  1
Delta2Y #3 :  1
USDMXN LONG GREEN
Trade Placed:    USDMXN     Short    TP:   22.483952809204926    SL:   22.398523595397535
...Analysis Complete
Analyzing Markets...
Delta2Y #1 :  1
Delta2Y #2 :  1
Delta2Y #3 :  1
USDMXN LONG GREEN
Trade Placed:    USDMXN     Short    TP:   22.483352809204927    SL:   22.397923595397536
...Analysis Complete
Analyzing Markets...
Delta2Y #1 :  1
Delta2Y #2 :  1
Delta2Y #3 :  1
USDJPY LONG GREEN
Trade Placed:    USDJPY     Short    TP:   106.25109764949195    SL:   106.04845117525403
...Analysis Complete
Analyzing Markets...
...Analysis Complete
Analyzing Markets...
Delta2Y #1 :  1
Delta2Y #2 :  1
Delta2Y #3 :  1
USDCHF LONG GREEN
Trade Placed:    USDCHF     Short    TP:   0.9186673010856582    SL:   0.9166463494571709
...Analysis Complete
Analyzing Markets...
Delta2Y #1 :  1
Delta2Y #2 : 

KeyboardInterrupt: 